In [ ]:
import numpy as np

### 50.000 combinaciones de lotería. Objetivo: Análisis de rendimiento del programa.

___

In [ ]:
combinaciones = 50000
n_bolas = 5

### Solo usando modulos del lenguaje 

In [ ]:
import random

In [ ]:
def sacar_bola_slow(combi, n_bola):
    "esta forma pued creo que puede producir bolas repetidas"
    bola = random.randint(0, 50)
    for bola_comp in range(n_bola):
        if bola == combi[bola_comp]:
            bola = random.randint(0, 50)
    return bola

In [ ]:
%%time
combi_ganadora = []
for i in range(n_bolas):
    combi_ganadora.append(sacar_bola_slow(combi_ganadora, i))

apuestas = []
aciertos = []

for combinancion in range(combinaciones):
    combi_apostada = []
    for n_bola in range(n_bolas):
        combi_apostada.append(sacar_bola_slow(combi_apostada, n_bola))
    apuestas.append(combi_apostada)
    
    aciertos_combinacion = 0
    for bola_ganadora in combi_ganadora:
        for bola_apostada in combi_apostada:
            if bola_ganadora == bola_apostada:
                aciertos_combinacion += 1
    aciertos.append(aciertos_combinacion)

for num_aciertos in range(n_bolas):
    print(f"{num_aciertos} : {aciertos.count(num_aciertos)}")

### Podemos mejorar la funcion sacar bola con recursion

In [ ]:
def sacar_bola(combi):
    "usa recursion, mo produce repetias"
    bola = random.randint(0, 50)
    if bola in combi:
        bola = sacar_bola(combi)
    return bola

In [ ]:
%%time
combi_ganadora = []
for i in range(n_bolas):
    combi_ganadora.append(sacar_bola(combi_ganadora))

apuestas = []
aciertos = []

for combinancion in range(combinaciones):
    combi_apostada = []
    for n_bola in range(n_bolas):
        combi_apostada.append(sacar_bola(combi_apostada))
    apuestas.append(combi_apostada)
    
    aciertos_combinacion = 0
    for bola_ganadora in combi_ganadora:
        for bola_apostada in combi_apostada:
            if bola_ganadora == bola_apostada:
                aciertos_combinacion += 1
    aciertos.append(aciertos_combinacion)

for num_aciertos in range(n_bolas):
    print(f"{num_aciertos} : {aciertos.count(num_aciertos)}")

#### Podemos sustituir por sets los dos bucles del final

In [ ]:
%%time
combi_ganadora = []
for i in range(5):
    combi_ganadora.append(sacar_bola(combi_ganadora))

apuestas = []
aciertos = []

for combinancion in range(combinaciones):
    combi_apostada = []
    for n_bola in range(5):
        combi_apostada.append(sacar_bola(combi_apostada))
    apuestas.append(combi_apostada)
    
    aciertos_combinacion = len(set(combi_apostada).intersection(combi_ganadora))
    aciertos.append(aciertos_combinacion)

for num_aciertos in range(5):
    print(f"{num_aciertos} : {aciertos.count(num_aciertos)}")

### Con numpy

In [ ]:
import numpy as np

In [ ]:
%%time
combinacion = np.random.choice(50, 5, replace=True)
combi_ganadora = np.zeros(50)
combi_ganadora[combinacion] = 1

aciertos = np.zeros(combinaciones)

for i in range(combinaciones):
    combinacion = np.random.choice(50, 5, replace=True)
    combi_apostada = np.zeros(50)
    combi_apostada[combinacion] = 1
    
    aciertos[i] = combi_ganadora@combi_apostada

unique, counts = np.unique(aciertos, return_counts=True)
print(f"{unique} {counts}")

Podemos definir las apuestas antes:

In [ ]:
combinacion = np.random.choice(50, 5, replace=True)
combi_ganadora = np.zeros(50)
combi_ganadora[combinacion] = 1

apuestas = np.zeros((combinaciones, 50))
apuestas[:, :5] = 1
[np.random.shuffle(apuesta) for apuesta in apuestas]

num_aciertos = apuestas @ combi_ganadora
unique, counts = np.unique(num_aciertos, return_counts=True)
print(f"{unique} {counts}")

aun así sigue siendo lento

### Lo mas eficiente que se puede hacer con numpy es de la siguiente forma:

In [ ]:
%%time
combinacion = np.random.choice(50, 5, replace=True)
combi_ganadora = np.zeros(50)
combi_ganadora[combinacion] = 1

apuestas = np.zeros((combinaciones, 50))
apuestas_num = np.random.rand(combinaciones, 50).argpartition(5,axis=1)[:,:5]
index = np.tile(np.expand_dims(np.arange(combinaciones), axis=0).transpose(), (1, 5))
apuestas[index, apuestas_num] = 1

num_aciertos = apuestas @ combi_ganadora
unique, counts = np.unique(num_aciertos, return_counts=True)

In [ ]:
print(f"{unique} {counts}")

Podemos medirlo varias veces

In [ ]:
%%timeit
combinacion = np.random.choice(50, 5, replace=True)
combi_ganadora = np.zeros(50)
combi_ganadora[combinacion] = 1

apuestas = np.zeros((combinaciones, 50))
apuestas_num = np.random.rand(combinaciones, 50).argpartition(5,axis=1)[:,:5]
index = np.tile(np.expand_dims(np.arange(combinaciones), axis=0).transpose(), (1, 5))
apuestas[index, apuestas_num] = 1

num_aciertos = apuestas @ combi_ganadora
unique, counts = np.unique(num_aciertos, return_counts=True)

133ms en mi ordenador aprox